# Сбор всех категорий Kaspi.kz

Скрипт собирает **все категории и подкатегории** со всех разделов Kaspi.kz и записывает в **ClickHouse**.

## Структура:
1. Импорты и настройки
2. Получение основных категорий (20 шт)
3. Функции для сбора подкатегорий
4. Сбор данных по всем категориям
5. Запись в ClickHouse

## 1. Импорты и настройки

In [ ]:
import pandas as pd
import requests
import time
import json
import urllib3

# Отключаем предупреждения SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Загружаем конфиг
with open('config.json', 'r') as f:
    CONFIG = json.load(f)

CH = CONFIG['clickhouse']

# Общие заголовки для Kaspi API
HEADERS = {
    'Accept': 'application/json, text/*',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,ru;q=0.7',
    'Connection': 'keep-alive',
    'Cookie': 'ks.tg=35; k_stat=d802b93e-0388-4fb8-b31c-c1c81eaf2645; kaspi.storefront.cookie.city=750000000',
    'Referer': 'https://kaspi.kz/shop/c/categories/?c=750000000',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36',
    'X-KS-City': '750000000',
    'sec-ch-ua': '"Google Chrome";v="143", "Chromium";v="143", "Not A(Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"'
}

print('Настройки загружены')
print(f'ClickHouse: {CH["CH_HOST"]}:{CH["CH_PORT"]}/{CH["CH_DB"]}.{CH["CH_TABLE"]}')

## 2. Функция записи в ClickHouse

In [ ]:
def insert_df(df: pd.DataFrame, table: str, database: str = None) -> int:
    """Записывает DataFrame в ClickHouse через HTTP интерфейс"""
    db = database or CH['CH_DB']
    url = f"http://{CH['CH_HOST']}:{CH['CH_PORT']}/"
    
    # Конвертируем DataFrame в JSONEachRow формат
    records = df.to_dict(orient='records')
    data = '\n'.join(json.dumps(r, ensure_ascii=False, default=str) for r in records)
    
    # POST запрос
    query = f"INSERT INTO {table} FORMAT JSONEachRow"
    
    r = requests.post(
        url,
        params={
            "database": db,
            "query": query
        },
        data=data.encode('utf-8'),
        auth=(CH['CH_USER'], CH['CH_PASS']),
        timeout=CH['CH_HTTP_TIMEOUT']
    )
    
    if r.status_code != 200:
        raise Exception(f"ClickHouse error: {r.text}")
    
    return len(records)

print('Функция insert_df загружена')

## 3. Получение основных категорий

In [ ]:
url = 'https://kaspi.kz/yml/main-navigation/n/n/desktop-menu'
params = {
    'depth': '1',
    'city': '750000000',
    'rootType': 'desktop'
}

response = requests.get(url, params=params, headers=HEADERS, verify=False, timeout=10)
print(f'Status: {response.status_code}')

main_data = response.json()
main_categories_df = pd.DataFrame(main_data['subNodes'])[['code', 'title', 'popularity']]
main_categories_df = main_categories_df.sort_values('popularity', ascending=False).reset_index(drop=True)

print(f'\nНайдено {len(main_categories_df)} основных категорий:\n')
main_categories_df

## 4. Функции для сбора данных

In [ ]:
def get_category_tree(category_code):
    """Получает дерево подкатегорий для указанной категории"""
    url = 'https://kaspi.kz/yml/product-view/pl/filters'
    params = {
        'q': f':category:{category_code}:availableInZones:Magnum_ZONE1',
        'text': '',
        'all': 'false',
        'sort': 'relevance',
        'ui': 'd',
        'i': '-1',
        'c': '750000000'
    }
    
    response = requests.get(url, params=params, headers=HEADERS, verify=False, timeout=10)
    response.raise_for_status()
    
    data = response.json()
    return data.get('data', {}).get('treeCategory', {})


def flatten_categories(items, main_category_code, main_category_title, parent_id=None, parent_title=None, level=0):
    """Рекурсивно разворачивает вложенные категории в плоский список"""
    rows = []
    
    for item in items:
        row = {
            'main_category_code': main_category_code,
            'main_category_title': main_category_title,
            'id': item.get('id', ''),
            'title': item.get('title', ''),
            'title_ru': item.get('titleRu', ''),
            'link': item.get('link', ''),
            'active': item.get('active', False),
            'count': item.get('count', 0),
            'popularity': item.get('popularity', 0),
            'expanded': item.get('expanded', False),
            'parent_id': parent_id,
            'parent_title': parent_title,
            'level': level
        }
        rows.append(row)
        
        # Рекурсивно обрабатываем подкатегории
        if 'items' in item and item['items']:
            child_rows = flatten_categories(
                item['items'],
                main_category_code=main_category_code,
                main_category_title=main_category_title,
                parent_id=item.get('id'),
                parent_title=item.get('title'),
                level=level + 1
            )
            rows.extend(child_rows)
    
    return rows

print('Функции загружены')

## 5. Сбор данных по всем категориям

Проходим по каждой из 20 основных категорий и собираем все подкатегории.

In [ ]:
# Список основных категорий
main_categories = []
for node in main_data['subNodes']:
    main_categories.append({
        'code': node['code'],
        'title': node['title']
    })

# Собираем все данные
all_rows = []
errors = []

print(f'Начинаем сбор данных по {len(main_categories)} категориям...\n')
print('-' * 60)

for i, cat in enumerate(main_categories, 1):
    code = cat['code']
    title = cat['title']
    
    print(f'[{i:2d}/{len(main_categories)}] {title} ({code})...', end=' ')
    
    try:
        tree = get_category_tree(code)
        
        # Обрабатываем items
        if 'items' in tree and tree['items']:
            rows = flatten_categories(tree['items'], code, title)
            all_rows.extend(rows)
            print(f'OK ({len(rows)} подкатегорий)')
        else:
            print('OK (нет подкатегорий)')
            
    except Exception as e:
        print(f'ОШИБКА: {e}')
        errors.append({'category': code, 'error': str(e)})
    
    # Задержка между запросами
    if i < len(main_categories):
        time.sleep(0.5)

print('-' * 60)
print(f'\nГотово! Собрано {len(all_rows)} подкатегорий')
if errors:
    print(f'Ошибок: {len(errors)}')

## 6. Финальный датасет

In [ ]:
# Создаём итоговый DataFrame
df = pd.DataFrame(all_rows)

# Приводим типы для ClickHouse
df['active'] = df['active'].astype(bool)
df['expanded'] = df['expanded'].astype(bool)
df['count'] = df['count'].astype(int)
df['popularity'] = df['popularity'].astype(int)
df['level'] = df['level'].astype(int)

# Заполняем None строками для ClickHouse
df['parent_id'] = df['parent_id'].fillna('')
df['parent_title'] = df['parent_title'].fillna('')

print(f'Итоговый датасет: {len(df)} строк, {len(df.columns)} колонок')
print(f'\nКолонки: {list(df.columns)}')
df.head(10)

## 7. Запись в ClickHouse

In [ ]:
# Записываем в ClickHouse
table = CH['CH_TABLE']
db = CH['CH_DB']

print(f'Записываем {len(df)} строк в {db}.{table}...')

try:
    inserted = insert_df(df, table, db)
    print(f'Успешно записано {inserted} строк в ClickHouse!')
except Exception as e:
    print(f'Ошибка записи: {e}')

## 8. Просмотр датасета

In [ ]:
df